In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ankylosing_Spondylitis"
cohort = "GSE73754"

# Input paths
in_trait_dir = "../../input/GEO/Ankylosing_Spondylitis"
in_cohort_dir = "../../input/GEO/Ankylosing_Spondylitis/GSE73754"

# Output paths
out_data_file = "../../output/preprocess/Ankylosing_Spondylitis/GSE73754.csv"
out_gene_data_file = "../../output/preprocess/Ankylosing_Spondylitis/gene_data/GSE73754.csv"
out_clinical_data_file = "../../output/preprocess/Ankylosing_Spondylitis/clinical_data/GSE73754.csv"
json_path = "../../output/preprocess/Ankylosing_Spondylitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Sexual Dimorphism in the Th17 Signature of Ankylosing Spondylitis"
!Series_summary	"Male AS patients have an elevated Th17 cell frequency vs. female AS patients (Gracey et al, Arthritis and Rheumatology, 2015). This analysis was performed to further examine differences between male and female AS patients"
!Series_overall_design	"AS patients were compared to healthy controls (HC). For sex-specific anaylsis, three groups were compared: F-HC vs. M-HC, M-AS vs. M-HC and F-AS vs. F-HC. A one way ANOVA was performed to identify genes differentially regulated in male and female AS patients"
Sample Characteristics Dictionary:
{0: ['Sex: Male', 'Sex: Female'], 1: ['age (yr): 53', 'age (yr): 26', 'age (yr): 29', 'age (yr): 50', 'age (yr): 35', 'age (yr): 48', 'age (yr): 18', 'age (yr): 39', 'age (yr): 49', 'age (yr): 43', 'age (yr): 59', 'age (yr): 51', 'age (yr): 45', 'age (yr): 52', 'age (yr): 77', 'age (yr): 34', 'age (yr): 31', 'age (yr): 23', 'age (yr)

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Dict, Any, Optional, Callable
import json

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a gene expression study
# comparing AS patients to healthy controls with a focus on sexual dimorphism
is_gene_available = True

# 2.1 Data Availability
# Trait information is in row 3 'disease: Ankylosing Spondylitis' or 'disease: healthy control'
trait_row = 3

# Age information is in row 1 'age (yr): XX'
age_row = 1

# Gender information is in row 0 'Sex: Male' or 'Sex: Female'
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary (1 for AS, 0 for healthy control)"""
    if pd.isna(value):
        return None
    value = value.split(':', 1)[1].strip() if ':' in value else value.strip()
    if 'ankylosing spondylitis' in value.lower():
        return 1
    elif 'healthy control' in value.lower():
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous"""
    if pd.isna(value):
        return None
    value = value.split(':', 1)[1].strip() if ':' in value else value.strip()
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    value = value.split(':', 1)[1].strip() if ':' in value else value.strip()
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction - Note that we don't have the actual clinical data
# We can only proceed with initial validation at this stage
print(f"Initial validation for {cohort} completed.")
print(f"Gene expression data available: {is_gene_available}")
print(f"Trait data available: {is_trait_available}")
print(f"Trait conversion function and row identifier have been prepared.")
print(f"Age conversion function and row identifier have been prepared.")
print(f"Gender conversion function and row identifier have been prepared.")
print(f"To extract clinical features, the original clinical_data.csv file would be needed.")


Initial validation for GSE73754 completed.
Gene expression data available: True
Trait data available: True
Trait conversion function and row identifier have been prepared.
Age conversion function and row identifier have been prepared.
Gender conversion function and row identifier have been prepared.
To extract clinical features, the original clinical_data.csv file would be needed.


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene data dimensions: 47323 genes × 72 samples


### Step 4: Gene Identifier Review

In [5]:
# These are Illumina probe identifiers (ILMN_), not human gene symbols
# They need to be mapped to gene symbols for proper gene expression analysis
# ILMN_ prefixes indicate Illumina BeadArray platform probe IDs

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the correct columns for mapping from the gene annotation data
# ID column contains the probe IDs that match with gene_data indices
# Symbol column contains the gene symbols we want to map to

# 2. Get gene mapping using the appropriate columns from gene annotation
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the mapped gene expression data
print("\nAfter mapping to gene symbols:")
print(f"Gene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First 10 gene symbols:")
print(gene_data.index[:10])

# Save the processed gene data to file
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nProcessed gene expression data saved to {out_gene_data_file}")



After mapping to gene symbols:
Gene data dimensions: 21464 genes × 72 samples
First 10 gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')



Processed gene expression data saved to ../../output/preprocess/Ankylosing_Spondylitis/gene_data/GSE73754.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols in the gene expression data...")
# From the previous step output, we can see the data already contains gene symbols
# like 'A1BG', 'A1CF', 'A2M' which need to be normalized
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene data shape: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print(f"Normalized gene data shape: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Extract clinical features from scratch instead of loading the empty file
print("Extracting clinical features from original clinical data...")
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row,
    convert_trait,
    age_row,
    convert_age,
    gender_row,
    convert_gender
)

# Save the extracted clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

print("Clinical features preview:")
print(preview_df(clinical_features))

# Check if clinical features were successfully extracted
if clinical_features.empty:
    print("Failed to extract clinical features. Dataset cannot be processed further.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Clinical features could not be extracted from the dataset."
    )
    print("Dataset deemed not usable due to lack of clinical features.")
else:
    # 2. Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")

    # 3. Handle missing values systematically
    linked_data = handle_missing_values(linked_data, trait_col=trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")

    # 4. Check if the dataset is biased
    print("\nChecking for bias in feature variables:")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

    # 5. Conduct final quality validation
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data for aniridia patients and healthy controls."
    )

    # 6. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols in the gene expression data...
Original gene data shape: 21464 genes × 72 samples
Normalized gene data shape: 20259 genes × 72 samples


Normalized gene expression data saved to ../../output/preprocess/Ankylosing_Spondylitis/gene_data/GSE73754.csv
Extracting clinical features from original clinical data...
Clinical features saved to ../../output/preprocess/Ankylosing_Spondylitis/clinical_data/GSE73754.csv
Clinical features preview:
{'GSM1902130': [1.0, 53.0, 1.0], 'GSM1902131': [1.0, 26.0, 1.0], 'GSM1902132': [1.0, 29.0, 1.0], 'GSM1902133': [1.0, 50.0, 1.0], 'GSM1902134': [1.0, 35.0, 1.0], 'GSM1902135': [1.0, 48.0, 1.0], 'GSM1902136': [1.0, 18.0, 1.0], 'GSM1902137': [1.0, 39.0, 1.0], 'GSM1902138': [1.0, 49.0, 1.0], 'GSM1902139': [1.0, 43.0, 1.0], 'GSM1902140': [1.0, 43.0, 1.0], 'GSM1902141': [1.0, 18.0, 1.0], 'GSM1902142': [1.0, 59.0, 1.0], 'GSM1902143': [1.0, 51.0, 1.0], 'GSM1902144': [1.0, 18.0, 1.0], 'GSM1902145': [1.0, 45.0, 1.0], 'GSM1902146': [1.0, 52.0, 1.0], 'GSM1902147': [1.0, 77.0, 1.0], 'GSM1902148': [1.0, 34.0, 1.0], 'GSM1902149': [1.0, 31.0, 1.0], 'GSM1902150': [1.0, 51.0, 1.0], 'GSM1902151': [1.0, 23.0, 1.

Data shape after handling missing values: (72, 20262)

Checking for bias in feature variables:
For the feature 'Ankylosing_Spondylitis', the least common label is '0.0' with 20 occurrences. This represents 27.78% of the dataset.
The distribution of the feature 'Ankylosing_Spondylitis' in this dataset is fine.

Quartiles for 'Age':
  25%: 28.75
  50% (Median): 41.5
  75%: 51.25
Min: 18.0
Max: 77.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 35 occurrences. This represents 48.61% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Ankylosing_Spondylitis/GSE73754.csv
